In [3]:
import os

from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=[{'role': 'user', 'content': 'hi'}],
  temperature=0.0,
)

print(completion.choices[0].message.content)



Hello! How can I assist you today?


In [4]:
import json

with open('./res/reviews.json') as f:
    review_list = json.load(f)


review_list[:3]

[{'review': '위치는 최상!!!  주차는 ㅠㅠ', 'stars': 4, 'date': '1일 전'},
 {'review': '호텔 위치가 안녕인사동 건물이라 찾기 쉽고 여행하기 너무 최적의 장소였어요! 그 건물 미피전시회 보러왔다가 너무 편하게 놀다가네요. 호텔 조식 좋아하는데 나쁘진않치만 근처에 먹을데가 너무 많아서 다음엔 신청 안할려구용 ㅋ 침대가 엄청 편하고 호텔이 너무 깨끗했습니다. 체크인이 오래걸리는 문제만 해결되면 재방문 의사있습니다!',
  'stars': 5,
  'date': '2일 전'},
 {'review': '위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요',
  'stars': 4,
  'date': '2일 전'}]

In [5]:
good_cnt, bad_cnt = 0, 0

for r in review_list:
  if r['stars'] == 5:
    good_cnt += 1
  else:
    bad_cnt += 1

good_cnt, bad_cnt

(99, 21)

In [7]:
reviews_good, reviews_bad = [], []

for r in review_list:
  if r['stars'] == 5:
    reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
  else:
    reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]위치는 최상!!!  주차는 ㅠㅠ[REVIEW_END]',
 '[REVIEW_START]위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요[REVIEW_END]',
 '[REVIEW_START]너무 번화가 쪽은 아니라서 밤에 조용히 쉬기 좋았어요[REVIEW_END]']

In [8]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]위치는 최상!!!  주차는 ㅠㅠ[REVIEW_END]\n[REVIEW_START]위치 굿 \n객실내 먼지가 좀 많고 건조함 \n뷰는 크게 기대하지 마세요[REV'

In [9]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
  with open(path) as f:
    review_list = json.load(f)

  reviews_good, reviews_bad = [], []

  current_date = datetime.datetime.now()
  date_boundary = current_date - datetime.timedelta(days=6*30)

  for r in review_list:
    review_date = r['date']
    try :
      review_date = parser.parse(review_date)
    except (ValueError, TypeError):
      review_date = current_date

    if review_date < date_boundary:
      continue

    if r['stars'] == 5:
      reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
      reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

  reviews_good_text = '\n'.join(reviews_good)
  reviews_bad_text = '\n'.join(reviews_bad)

  return reviews_good_text, reviews_bad_text


good, bad = preprocess_reviews()

good[:100]

'[REVIEW_START]호텔 위치가 안녕인사동 건물이라 찾기 쉽고 여행하기 너무 최적의 장소였어요! 그 건물 미피전시회 보러왔다가 너무 편하게 놀다가네요. 호텔 조식 좋아하는데 '

# 평가용 함수 작성

In [10]:
def pairwise_eval(reviews, answer_a, answer_b):
  eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: \"[[A]]\" if assistant A is better, \"[[B]]\" if assistant B is better, and \"[[C]]\" for a tie.

[User Reviews]
{reviews}
[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]
[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""
  
  completion = client.chat.completions.create(
    model='gpt-4o-2024-05-13',
    messages=[{'role': 'system', 'content': eval_prompt}],
    temperature=0.0,
  )

  return completion

## Baseline 모델 개발

In [11]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [12]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
  prompt = prompt + '\n\n' + reviews

  completion = client.chat.completions.create(
    model=model,
    messages=[{'role': 'system', 'content': prompt}],
    temperature=temperature,
  )

  return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

1. 인사동에 위치한 호텔로 여행하기에 최적의 장소이며, 깨끗하고 편안한 침대가 좋았다.
2. 항상 만족하는 최고의 위치와 객실 상태를 자랑하는 호텔이다.
3. 근처 맛집이 많아서 좋았고, 주변 동네가 예쁘고 다양한 관광지가 있었다.
4. 침구류가 깨끗하고 주차가 편리하며, 소음 없이 편안한 숙박을 즐길 수 있다.
5. 위치가 좋고 깨끗한 숙소 내부와 친절한 직원들이 만족스러운 숙박을 제공한다.


In [13]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주위에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [14]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A provides a more detailed and comprehensive summary of the user reviews. It highlights specific aspects such as the hotel's location, the comfort of the beds, the cleanliness, and the proximity to tourist attractions and amenities. Assistant A also mentions the issue with the check-in time and the positive feedback about the view and service, which are all relevant points from the reviews.

On the other hand, Assistant B's summary is more general and lacks the depth and specific details found in Assistant A's response. While it does mention the location, cleanliness, and friendly service, it does not capture the variety of specific feedback provided in the reviews.

Therefore, Assistant A's response is more helpful, relevant, and detailed.

[[A]]


# 대규모 평가 스크립트

- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt 에 대해 temperature 높여서 평가 진행

In [15]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['1. 인사동에 위치한 깨끗하고 편안한 호텔로, 객실 상태와 위치에 대한 만족도가 높음\n2. 객실이 깨끗하고 조용하여 휴식하기 좋은 호텔\n3. 주변 맛집이 많아 위치가 좋은 숙소\n4. 인사동 분위기를 느낄 수 있는 포근한 숙박이 가능한 곳\n5. 소파 테이블 부족 등 일부 편의성 부족 사항을 제외하면 전체적으로 만족도가 높음',
 '1. 인사동에 위치한 호텔로 여행하기에 최적의 장소로 인식되며, 깨끗한 객실과 편안한 침대를 갖추고 있음.\n2. 최고의 위치와 객실 상태로 항상 만족을 주는 숙소로 평가됨.\n3. 근처 맛집이 다수 위치해 있어 식사하기 좋은 장소로 인식됨.\n4. 깔끔하고 예쁜 주변 동네와 함께 편안한 숙박을 제공함.\n5. 청결하고 위치가 좋아 만족스러운 숙박 경험을 주는 인사동에 위치한 숙소로 추천됨.',
 '1. 인사동에 위치한 호텔로 위치가 편리하고 깔끔한 숙소.\n2. 호텔 컨디션과 위치에 만족하는 리뷰.\n3. 주변 맛집이 많아 위치가 좋다는 의견.\n4. 조용하고 깔끔한 객실로 편안한 휴식을 취할 수 있는 숙소.\n5. 호텔이 조용하고 깨끗하며 위치가 좋다는 찬사가 담긴 후기.',
 '1. 위치가 최적하고 인사동 건물 내에 위치한 호텔이며, 깨끗하고 친절한 서비스를 제공한다.\n2. 최고의 위치와 객실 상태로 항상 만족스러운 숙박을 경험할 수 있다.\n3. 근처 맛집이 다양하고 호텔은 깔끔하며 근사한 숙소이다.\n4. 깨끗하고 예쁜 주변 동네에서 편안한 숙박을 즐길 수 있다.\n5. 친절한 직원, 깨끗한 객실, 편리한 주차시설 등 다양한 편의시설을 갖춘 호텔이다.',
 '1. 위치가 좋아서 여행하기 최적이며 미피전시회 근처에 위치하여 놀거리가 풍부하다.\n2. 호텔은 깨끗하고 침대도 편안하며, 조식은 괜찮지만 주변 맛집이 많아서 재방문할 의사가 있다.\n3. 인사동 분위기를 느낄 수 있는 포근한 분위기의 숙소로 깔끔하며 좋은 위치에 위치해 있다.\n4. 친절한 직원과 깨끗한 객실이 인상적이며, 주변 관광 명소와의 접근성이 좋다

In [16]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
  a_cnt, b_cnt, draw_cnt = 0, 0, 0

  for i in tqdm(range(len(answers_a))):
    completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
    verdict_text = completion.choices[0].message.content
    
    if '[[A]]' in verdict_text:
      a_cnt += 1
    elif '[[B]]' in verdict_text:
      b_cnt += 1
    elif '[[C]]' in verdict_text:
      draw_cnt += 1
    else:
      print('Evaluation Error')

  
  return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.89s/it]

Wins: 10, Losses: 0, Ties: 0


## 모델 고도화 1 - 조건들 명시

In [17]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:18<00:00,  1.80s/it]

Wins: 10, Losses: 0, Ties: 0


## 모델 고도화 2 - 입력 데이터의 품질 증가

In [18]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [19]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:23<00:00,  2.32s/it]

Wins: 10, Losses: 0, Ties: 0


## 모델 고도화 3 - Few-Shot Prompting

In [20]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:25<00:00,  2.52s/it]

Wins: 9, Losses: 1, Ties: 0


In [21]:
summaries

['한여름 더위에 이른 오후, 안녕인사동 건물 속 감성피트니스 전시회를 즐기며 즐거운 시간을 보내셨던 손님께서 호텔의 편안한 조식과 편안한 침대, 청결한 객실에 만족하셨던 후기를 남기셨습니다. 체크인 속도 문제만 해결된다면 재방문의사가 있다고 합니다. 만족의 말들이어서 기분 좋은 한편인 것 같아요. 위치와 객실 상태 등이 언제나 만족스러웠던 것으로 드러나는 후기입니다. 조계사의 북소리가 크게 들렸지만 여전히 만족하셨다고 하네요. 위치와 서비스, 청결함 등에서 좋은 평을 받은 호텔입니다. 위치가 매우 편리하고 조계사 뷰를 즐기면서 편안한 휴식을 취하셨다는 훌륭한 후기를 남긴 손님도 있습니다. 위치와 친절한 직원, 깨끗한 객실 등 여러 면에서 만족스러운 숙박이었다고 작성한 후기도 있네요.기본 컨디션도 좋았다는 서운이며, 방음과 건조함을 언급하며 다소 아쉬워하는 후기도 있습니다. 호텔의 깔끔함과 위치, 객실 편안함으로 즐거운 여행을 보내신 후기도 있습니다. 인사동과 조계사를 즐기기에 딱 좋은 위치에 있었다는 만족스러운 후기도 있어요. 위치적인 장점이 매력적이라 단점이 더욱 상쇄된다는 긍정적인 평도 있지만, 가격이 조금 낮았으면 더 좋겠다는 솔직한 의견도 보입니다. 숙박 및 놀이를 즐기면서 만족하는 손님들 가운데 하나로 나타나는 좋은 후기입니다. 다양한 단소나 에러걸리나 사소한 불편한점 개선을 희망하는 후기도 있네요. 위치, 청결함, 주변 환경 등에 만족하셨던 손님이 호텔의 깨끗함과 주변 볼거리에 잠시 눈을 환기시킨 후 끝내기 좋다는 후기를 남기십니다.',
 '이 호텔은 안녕인사동 건물에 위치해 있어 찾기가 쉽고 여행하기에 최적의 장소입니다. 호텔 조식은 나쁘지 않지만 주변 맛집이 많아서 다른 식사장소를 고려할 정도로 편리합니다. 침대는 편안하고 호텔은 매우 깨끗합니다. 체크인 시간이 개선된다면 재방문 의사가 있는 호텔입니다. 이곳은 항상 만족하고 최고의 위치에 위치한 호텔로 늘 만족하는 곳입니다. 위치와 청결도로 만족한 급하게 예약한 것도 누리기 좋았습니다. 호텔은

In [22]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:25<00:00,  2.58s/it]

Wins: 8, Losses: 2, Ties: 0


In [23]:
summaries

['이 숙소는 인사동에 위치해 있어 관광하기에 최적이며, 객실 상태와 서비스에 대한 만족도가 높게 평가되었습니다. 다수의 리뷰에서 친절한 직원 응대와 깨끗한 객실을 강조하였으며, 주변 맛집과 먹거리가 풍부하다는 점이 언급되었습니다. 그러나 일부 리뷰에서는 체크인 속도나 소음에 대한 언급도 있었습니다. 전반적으로 재방문 의사가 높은 숙소로 평가됩니다.',
 '이 숙소는 주변 관광지와 가까운 위치에 있어 이동이 편리하며, 객실의 청결도가 높게 평가되었습니다. 직원들의 친절함과 서비스에 대한 만족도도 높았습니다. 다만 일부 리뷰에서는 체크인 과정이 다소 오래 걸린다는 언급이 있었지만, 전반적으로 재방문 의사가 높은 것으로 보입니다.',
 '이 숙소는 인사동에 위치하여 위치가 매우 우수하며 청결한 객실과 친절한 직원들이 많이 언급되고 있습니다. 또한 수영장 등 부대시설이 만족스럽다는 평가도 많이 받았습니다. 다만, 체크인 속도에 대한 불만이 일부 리뷰에서 언급되었으나, 전반적으로 아주 만족한 이용객이 많아 재방문 의사가 높은 숙소로 평가됩니다.',
 '이 숙소는 위치가 인사동에 있어 여행하기에 최적이며, 늘 만족스러운 객실 상태와 친절한 직원들로 평가받고 있습니다. 일부 소음이 문제가 될 수 있지만, 전반적으로 청결하고 편리한 숙소로 재방문 의사가 있는 이용객들이 많이 보고 있습니다. 부대시설과 위치면에서 좋은점을 강조하면서도, 체크인 시간의 지연이나 소음으로 인한 불편함에 대해 개선 요청이 있습니다. 다양한 먹거리와 구경할 곳이 주변에 많아 친절한 직원과 청결한 객실 등이 장점으로 인식되고 있습니다.',
 '이 숙소는 인사동에 위치하여 여행하기에 최적의 장소로 손꼽히고 있습니다. 객실 상태와 위치에 대한 만족도가 높게 평가되었으며, 조식은 나쁘지 않았지만 주변 맛집 다양성으로 인해 신청을 하지 않을 정도라는 의견도 있습니다. 침대의 편안함과 호텔 청결도에 대한 양호한 평가가 잇따르고 있습니다. 다만, 체크인 속도에 대한 불만 사항이 있어 해결되면 재방문의사가 있는 

In [24]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

In [25]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:23<00:00,  2.40s/it]

Wins: 10, Losses: 0, Ties: 0


In [26]:
summaries


['입력하신 리뷰들을 요약한 결과는 다음과 같습니다:\n\n이 숙소는 인사동 건물에 위치하여 여행하기에 최적의 장소로 평가받고 있습니다. 객실은 깨끗하고 침구도 편안하며, 위치가 편리하다는 평가를 받았습니다. 직원은 친절하며, 서울을 탐험하기에 이 숙소가 이상적이라고 말씀하셨습니다. 다만, 체크인 속도가 느리다는 결함이 지적되었습니다. 전반적으로 이용객들은 좋은 위치와 깨끗한 객실 상태로 만족하며, 재방문하고 싶다는 의사를 밝히고 있습니다. 추가로, 조계사 뷰를 선호하는 투숙객들이 있었으나, 소음이 좀 있다는 리뷰도 있었습니다. 전반적으로 숙박 경험에 만족한 리뷰가 많았습니다.',
 '요약 결과:\n이 숙소는 주변 관광지와의 접근성이 좋고, 깨끗한 객실과 친절한 직원 서비스로 많은 이용객들에게 만족을 준다. 고객들은 대체로 재방문 의사가 높으며, 조식이나 위치 등 다양한 요소에서 좋은 평가를 받는 편입니다. 그러나 체크인 속도나 약간의 소음, 온도와 먼지 문제 등 몇 가지 개선이 필요한 부분도 언급되어 있습니다. 종합적으로 위치와 객실의 깨끗함, 서비스에 대한 양호한 평가로 추천할만한 숙소로 평가됩니다.',
 '이 숙소는 위치가 인사동과 가까워서 여행하기에 최적의 장소로 인기가 높습니다. 객실 상태는 깨끗하고 침구도 편안하며, 체크인이 오래 걸리는 점을 빼면 재방문을 원하는 이용객이 많습니다. 주변 맛집이 많아 조식은 선택지가 많은 편이고, 조계사 뷰와 인사동 뷰를 즐길 수 있는 낮은 층수부터 고층 뷰까지 다양한 선택지를 제공합니다. 다만 소음에 조금 민감한 분들은 조심해야할 수도 있습니다. 호텔 직원들의 친절한 응대와 편리한 인프라, 그리고 주변 관광지 접근성 등이 좋은 평가를 받고 있습니다.',
 '이 숙소는 위치가 인사동 중심이라 여행하기에 최적이었고, 호텔 조식은 만족스러웠지만 인근 먹거리가 다양해서 조식을 추가로 구매할 의향이 적다는 리뷰가 있습니다. 객실은 깨끗하고 침대가 편안하며, 첫 방문 시 체크인 시간이 길어 문제를 겪었지만 바로 해결되면 다시